In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

folder_path = '/content/drive/My Drive/'
ratings_path = f'{folder_path}/ratings.csv'

ratings = pd.read_csv(ratings_path, sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

folder_path = '/content/drive/My Drive/'
movies_path = f'{folder_path}/movies.csv'
movies = pd.read_csv(movies_path, sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [4]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [6]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [8]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:
def genre_recommendations(title, n=10):
    idx = movies[movies['title'] == title].index[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores_sort = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores_lim = sim_scores_sort[1:n+1]
    movie_indices = [i[0] for i in sim_scores_lim]
    return movies.iloc[movie_indices][['title']]

In [12]:
# Example
genre_recommendations('GoldenEye (1995)')

,title
345,Clear and Present Danger (1994)
543,Surviving the Game (1994)
724,"Rock, The (1996)"
788,Daylight (1996)
825,Chain Reaction (1996)
978,Maximum Risk (1996)
1467,Anaconda (1997)
1513,Con Air (1997)
1693,Firestorm (1998)
3686,"Perfect Storm, The (2000)"



### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
data = pd.merge(ratings, movies, on='movie_id')

In [15]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [16]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [17]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [18]:
def get_top_n_similar_users(user_id, n=5):
    user_genre_matrix = data.pivot_table(
        index='user_id',
        columns='genres',
        values='rating',
        aggfunc='mean'
        )    .    fillna(0)

    user_similarity = cosine_similarity(user_genre_matrix)

    user_similarity_df = pd.DataFrame(
        user_similarity,
        index=user_genre_matrix.index,
        columns=user_genre_matrix.index
        )

    sim_scores = user_similarity_df[user_id]
    sim_scores = sim_scores.sort_values(ascending=False)

    return sim_scores.iloc[1:n+1].index


In [19]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):

    similar_users = get_top_n_similar_users(user_id, n_similar_users)
    similar_users_movies = data[data['user_id'].isin(similar_users)]

    target_user_movies = data[data['user_id'] == user_id]['movie_id']
    recommended_movies = similar_users_movies[~similar_users_movies['movie_id'].isin(target_user_movies)]

    top_n_recommended_movies = recommended_movies.groupby('movie_id').size().sort_values(ascending=False).head(n_recommendations).index

    return movies[movies['movie_id'].isin(top_n_recommended_movies)][['title', 'genres']]

In [20]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [21]:
recommended_movies

,title,genres
453,"Fugitive, The (1993)","['Action', 'Thriller']"
589,"Silence of the Lambs, The (1991)","['Drama', 'Thriller']"
999,Old Yeller (1957),"[""Children's"", 'Drama']"
1010,Winnie the Pooh and the Blustery Day (1968),"['Animation', ""Children's""]"
1207,Amadeus (1984),['Drama']
1575,L.A. Confidential (1997),"['Crime', 'Film-Noir', 'Mystery', 'Thriller']"
2012,"Little Mermaid, The (1989)","['Animation', ""Children's"", 'Comedy', 'Musical..."
2789,American Beauty (1999),"['Comedy', 'Drama']"
3003,Moonstruck (1987),['Comedy']
3352,Animal House (1978),['Comedy']


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [22]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [23]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# calculate rating matrix

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

data = pd.merge(ratings, movies, on='movie_id')

user_item_matrix = data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

user_item_matrix_sparse = csr_matrix(user_item_matrix.values)


In [48]:
U, sigma, Vt = svds(user_item_matrix_sparse, k=50)
sigma = np.diag(sigma)

In [34]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [40]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index)

preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,4.292556,0.164772,-0.184504,-0.018528,0.021516,-0.181793,-0.102936,0.157111,-0.058523,-0.164231,...,0.032475,0.007048,0.033170,-0.007577,-0.075329,0.394578,0.124714,0.051545,0.057349,0.076743
2,0.754817,0.128981,0.341128,0.009540,0.001830,1.314580,0.077427,0.062218,0.163983,1.514572,...,-0.051376,-0.015780,-0.010918,0.055628,-0.016037,0.166941,-0.421144,-0.106675,-0.049650,-0.126564
3,1.844858,0.473855,0.098573,-0.039309,-0.019895,-0.154038,-0.141531,0.111551,0.035977,0.738427,...,0.049994,0.003596,0.022734,0.043377,0.034825,0.120205,0.086553,0.034783,0.029337,-0.121871
4,0.395484,-0.045487,0.033716,0.083890,0.051561,0.260676,-0.081541,0.023891,0.051932,-0.077170,...,0.010847,0.007518,0.004269,0.009527,-0.079630,0.077845,0.051369,-0.017046,0.018980,-0.047580
5,1.557609,-0.006700,-0.044860,0.249014,-0.043320,1.518140,-0.164306,-0.043626,-0.078771,0.424064,...,0.097708,0.013847,-0.024256,-0.043126,-0.065997,-0.025571,0.517685,0.008508,0.106630,0.221219


In [52]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    sorted_user_predictions = preds_df.loc[user_id].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == user_id]
    user_full = user_data.merge(movies, how='left', on='movie_id').sort_values(['rating'], ascending=False)

    print(f'User {user_id} has already rated {user_full.shape[0]} movies.')
    print('Recommending the highest predicted rating movies not already rated.')

    # print(sorted_user_predictions.head())

    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
                       merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                             left_on='movie_id',
                             right_on='movie_id').
                       rename(columns={user_id: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1])

    return user_full, recommendations


In [53]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 1 has already rated 53 movies.
Recommending the highest predicted rating movies not already rated.


In [54]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [55]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
46,1,1029,5,0,1028,Dumbo (1941),"['Animation', ""Children's"", 'Musical']"
40,1,1,5,0,0,Toy Story (1995),"['Animation', ""Children's"", 'Comedy']"
18,1,3105,5,0,3104,Awakenings (1990),['Drama']
41,1,1961,5,0,1960,Rain Man (1988),['Drama']
23,1,527,5,0,526,Schindler's List (1993),"['Drama', 'War']"
37,1,1022,5,0,1021,Cinderella (1950),"['Animation', ""Children's"", 'Musical']"
14,1,1035,5,0,1034,"Sound of Music, The (1965)",['Musical']
25,1,48,5,0,47,Pocahontas (1995),"['Animation', ""Children's"", 'Musical', 'Romance']"
45,1,1028,5,0,1027,Mary Poppins (1964),"[""Children's"", 'Comedy', 'Musical']"


#### Evaluation

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=50)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns, index=train_ratings.index)


def predict_rating(user_id, movie_id):
    try:
        pred = preds_train.loc[user_id, movie_id]
    except KeyError:
        pred = np.nan
    return pred

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 2.740466076140738


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [63]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.users = torch.tensor(ratings['user_idx'].values, dtype=torch.long)
        self.movies = torch.tensor(ratings['movie_idx'].values, dtype=torch.long)
        self.ratings = torch.tensor(ratings['rating'].values, dtype=torch.float32)


    def __len__(self):
        #TODO
        return len(self.ratings)


    def __getitem__(self, idx):
        #TODO
        return self.users[idx], self.movies[idx], self.ratings[idx]


# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        #TODO
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)

        self.user_embedding = nn.Embedding(num_users, embedding_size)

        self.fc1 = nn.Linear(embedding_size * 2, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, user, movie):
        #TODO
        user_embedded = self.user_embedding(user)

        movie_embedded = self.movie_embedding(movie)

        x = torch.cat([user_embedded, movie_embedded], dim=-1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


# Create the dataset and dataloader

folder_path = '/content/drive/My Drive/'
ratings_path = f'{folder_path}/ratings.csv'

ratings = pd.read_csv(ratings_path, sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

folder_path = '/content/drive/My Drive/'
movies_path = f'{folder_path}/movies.csv'
movies = pd.read_csv(movies_path, sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

users = ratings['user_id'].unique()
movies = ratings['movie_id'].unique()

user_to_idx = {old_id: new_id for new_id, old_id in enumerate(users)}
movie_to_idx = {old_id: new_id for new_id, old_id in enumerate(movies)}

ratings['user_idx'] = ratings['user_id'].map(user_to_idx)
ratings['movie_idx'] = ratings['movie_id'].map(movie_to_idx)

dataset = RatingsDataset(ratings)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function and optimizer
num_users = len(users)
num_movies = len(movies)

model = RecommenderNet(num_users, num_movies)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for users, movies, ratings in dataloader:
        optimizer.zero_grad()
        outputs = model(users, movies).squeeze()
        loss = criterion(outputs, ratings)
        loss.backward()
        optimizer.step()
    print(f'eeepoch {epoch+1}/{num_epochs}, loss: {loss.item()}')

# Evaluation

model.eval()
with torch.no_grad():
    predictions = []
    targets = []
    for users, movies, ratings in dataloader:
        outputs = model(users, movies).squeeze()
        predictions.extend(outputs.tolist())
        targets.extend(ratings.tolist())

rmse = sqrt(mean_squared_error(targets, predictions))
print(f'Root Mean Squared Error: {rmse}')


eeepoch 1/10, loss: 1.1563386917114258
eeepoch 2/10, loss: 0.7711595296859741
eeepoch 3/10, loss: 0.7213550209999084
eeepoch 4/10, loss: 0.49034854769706726
eeepoch 5/10, loss: 0.40027764439582825
eeepoch 6/10, loss: 0.975921094417572
eeepoch 7/10, loss: 0.7213989496231079


In [ ]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

In [ ]:
def predict_unrated_movies(user_id, model, ratings, movies):
    #TODO
    us_rat_movies = ratings[ratings['user_id'] == user_id]['movie_id'].tolist()

    all_movies = movies['movie_id'].tolist()
    unrated_movies = list(set(all_movies) - set(us_rat_movies))

    user_tensor = torch.tensor([user_id] * len(unrated_movies), dtype=torch.long)
    movie_tensor = torch.tensor(unrated_movies, dtype=torch.long)
    with torch.no_grad():
        predictions = model(user_tensor, movie_tensor).squeeze().tolist()

    movie_ratings = list(zip(unrated_movies, predictions))
    movie_ratings.sort(key=lambda x: x[1], reverse=True)
    return movie_ratings[:10]


# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                             title             genres  \
1235      3338            For All Mankind (1989)        Documentary   
227       1704          Good Will Hunting (1997)              Drama   
1792      1797                    Everest (1998)        Documentary   
114        318  Shawshank Redemption, The (1994)              Drama   
2638       669                  Aparajito (1956)              Drama   
100        920         Gone with the Wind (1939)  Drama|Romance|War   
3341       598            Window to Paris (1994)             Comedy   
497        428              Bronx Tale, A (1993)              Drama   
74        1198    Raiders of the Lost Ark (1981)   Action|Adventure   
3359      2503           Apple, The (Sib) (1998)              Drama   

      predicted_rating  
1235          4.692185  
227           4.664774  
1792          4.663356  
114           4.659920  
2638          4.656652  
100           4.645355  
3341          4.635198  
497           4.63

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [62]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt


import pandas as pd

# folder_path =
# ratings_path = f'{folder_path}/'

# ratings = pd.read_csv(ratings_path, sep='\t', encoding='latin-1',
#                       usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

file_path = '/content/drive/My Drive/gmm-dataset.csv'
df = pd.read_csv(file_path)


click_matrix = df.set_index('URL')

click_matrix_normalized = click_matrix.div(click_matrix.sum(axis=1), axis=0)


gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(click_matrix_normalized)

log_likelihood = gmm.score(click_matrix_normalized)
bic = gmm.bic(click_matrix_normalized)
aic = gmm.aic(click_matrix_normalized)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: 19.719674033394806
BIC: -3036.716280039307
AIC: -3549.934806678961
